In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import random
!pip install tsplib95
import tsplib95
problem1 = tsplib95.load_problem('/content/drive/My Drive/SC_HW5/ali535.tsp')
problem2 = tsplib95.load_problem('/content/drive/My Drive/SC_HW5/d2103.tsp')
problem3 = tsplib95.load_problem('/content/drive/My Drive/SC_HW5/bayg29.tsp')


In [0]:
def init(problem,parentsnum):
  L=problem.dimension #city number
  parents=np.zeros([parentsnum,L])
  sequence = [i+1 for i in range(L)]
  for i in range(parentsnum):
    np.random.shuffle(sequence)
    parents[i,:]=sequence
  return parents

def cost(problem,seq):  
  L=problem.dimension
  cost=np.zeros([seq.shape[0],1])
  for i in range(seq.shape[0]):
    for j in range(L):
      if j==L-1:
        cost[i]+=problem.wfunc(int(seq[i,j]),int(seq[i,0]))
        break
      cost[i]+=problem.wfunc(int(seq[i,j]),int(seq[i,j+1]))  
  return cost

def tournament(problem,seq,t,k):
  sequence = [i for i in range(seq.shape[0])]
  out=np.zeros([k,seq.shape[1]])
  for i in range(k):
    subset = seq[random.sample(sequence, t)]
    subsetcost=cost(problem,subset)
    out[i]=subset[np.argmin(subsetcost)]
  return out

def cx(seqa,seqb):
  ca=cxrec(seqa,seqb)
  cb=cxrec(seqb,seqa)
  return ca,cb

def cxrec(seqa,seqb):
  ca=np.zeros([1,seqa.shape[0]])
  dicti={}
  k=0
  while(np.any(ca==0)):
    dicti['posa'+str(k)]=[]
    start=np.nonzero(ca==0)[1][0]
    posa=start
    dicti['posa'+str(k)].append(posa)
    while(1):
      posa=np.nonzero(seqa==seqb[posa])[0][0]
      if posa == dicti['posa'+str(k)][0]:
        break
      dicti['posa'+str(k)].append(posa)
    if k%2==0:
      for i in range(len(dicti['posa'+str(k)])):
        ca[0,dicti['posa'+str(k)][i]]=seqa[dicti['posa'+str(k)][i]]
    else:
      for i in range(len(dicti['posa'+str(k)])):
        ca[0,dicti['posa'+str(k)][i]]=seqb[dicti['posa'+str(k)][i]]
    k+=1
  return ca

def mutation(seq,rate):
  for i in range(seq.shape[0]):
    if np.random.rand()<=rate:
      a=np.random.randint(seq.shape[1])
      b=np.random.randint(seq.shape[1])
      if b<a:
        a,b=b,a
      elif a==b:
        continue
      temp=seq[i,b]
      seq[i,a+2:b+1]=seq[i,a+1:b]
      seq[i,a+1]=temp
  return seq

In [0]:
def genetic(problem,mutationrate=.02,parentsnum=50,t=3,repperc=.2,prep=.95,genlimit=1000,epslimit=20,epsilon=.01):
  gen=0
  eps=0
  repval=int(repperc*parentsnum)
  L=problem.dimension
  parents=init(problem,parentsnum)
  #parents[-1,:]=bestroute
  while(1):
    np.random.shuffle(parents)
    selectedparents=tournament(problem,parents,t,parentsnum)
    childs=np.zeros([parentsnum**2,L])
    _k=0
    for i in selectedparents:
      for j in selectedparents:
        if np.random.rand()<=prep:
          childs[_k,:]=cxrec(i,j)
          _k+=1
    childs = childs[0:_k,:]
    oldparents=parents
    childs=mutation(childs,mutationrate)
    parents=np.append(parents,cost(problem,parents),axis=1)
    childs=np.append(childs,cost(problem,childs),axis=1)
    childs=childs[childs[:, -1].argsort()]
    temp=parents
    temp=temp[temp[:, -1].argsort()]
    temp[-repval:,:]=childs[:repval,:]
    parents=temp[:parentsnum,:L]
    #allpop=np.append(childs,parents,axis=0)
    #allpop=allpop[allpop[:, -1].argsort()]
    #parents=allpop[allpop[:, -1].argsort()][:parentsnum,:L]
    if (cost(problem,oldparents[0,:].reshape([1,L]))-cost(problem,parents[0,:].reshape([1,L])))<epsilon:
      eps+=1
    else:
      eps=0
    best=parents[0,:]
    gen+=1
    if gen>=genlimit:
      break
    if eps>=epslimit:
      break
  print(gen)
  print(temp[0,L])
  print(temp[-1,L])
  return parents
    

In [0]:
genetic(problem1,mutationrate=.05,parentsnum=100,t=3,repperc=.2,prep=.95,genlimit=10000,epslimit=30,epsilon=1)